Initial baseline update & reward

*   Read current labels from env_work.FrankenData.dist_label.
*   Convert labels to an action matrix with labels_to_action.

Call env_work.step(action_curr). This runs env.step


The returned reward_curr becomes the baseline reward for Metropolis comparisons. This also ensures env_work reflects the post-step opinions (so proposals are compared against a consistent oldX).

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
import gymnasium as gym
from gymnasium import spaces
import gerry_environment
from copy import deepcopy
from typing import Optional, Tuple, Dict, Any
import math

In [ ]:
def labels_to_action(labels, num_districts, dtype=np.float32):
    """
    Convert an integer label vector (shape [N]) to an action matrix
    expected by env.step: shape (N, num_districts), each row is
    a 1-hot encoding of the desired district for that voter.
    """
    N = len(labels)
    A = np.zeros((N, num_districts), dtype=dtype)
    for i, lab in enumerate(labels):
        if lab >= 0 and lab < num_districts:
            A[i, int(lab)] = 1.0
        else:
            # keep row zeros -> will become -1 label in env.step (avoid if possible)
            pass
    return A

#**propose_flip**

The function implements a two-district cluster flip closely following the paper’s steps:

Choose a boundary/adjacent district pair (pick a cut edge).

Construct monochromatic internal edges for those two districts and optionally keep each internal edge with probability p_keep (this implements the Swendsen–Wang style random edge labeling).

Find the connected component (cluster) containing the chosen seed node u.

Flip that whole cluster to the other neighboring district.

Validate the proposed labels (no empty districts, contiguity) and reject if invalid

In [ ]:
def propose_flip(labels, geo_edge, rng, p_keep=1):

    N = len(labels)

    # collect cut edges
    #collect all cut edges (edges whose endpoints belong to different districts).
    # If there are no cut edges, there's nowhere to flip — return a copy of the current labels (no-op).
    diff_edges = [(int(u), int(v)) for u,v in geo_edge.T if labels[int(u)] != labels[int(v)]]
    if not diff_edges:
        return labels.copy()

    # randomly choose one cut edge (u,v) and record the two district labels dA (label at u) and dB (label at v).
    u, v = diff_edges[rng.integers(len(diff_edges))]
    dA, dB = labels[u], labels[v]

    # Build monochromatic adjacency only (edges where labels[x] == labels[y] and in {dA,dB})
    adj = [[] for _ in range(N)]
    for x,y in geo_edge.T:
        x,y = int(x), int(y)
        if labels[x] in (dA,dB) and labels[y] in (dA,dB) and labels[x] == labels[y]:
            if rng.random() < p_keep:
                adj[x].append(y)
                adj[y].append(x)

    # BFS to get cluster. find connected component of u (this component will be monochromatic because edges are monochromatic)
    cluster = set()
    stack = [u]
    while stack:
        node = stack.pop()
        if node in cluster:
            continue
        cluster.add(node)
        stack.extend(adj[node])

    cluster_label = labels[next(iter(cluster))]
    target = dB if cluster_label == dA else dA

    # flip cluster
    new_labels = labels.copy()
    for node in cluster:
        new_labels[node] = target

    # no empty districts
    for d in (dA, dB):
        if not np.any(new_labels == d):
            return labels.copy()

    # contiguity check for affected districts
    def is_contiguous(lab):
        nodes = [i for i in range(N) if new_labels[i] == lab]
        if not nodes:
            return False
        seen = set([nodes[0]])
        stack = [nodes[0]]
        while stack:
            n = stack.pop()
            for x,y in geo_edge.T:
                x,y = int(x), int(y)
                if x==n and new_labels[y]==lab and y not in seen:
                    seen.add(y); stack.append(y)
                elif y==n and new_labels[x]==lab and x not in seen:
                    seen.add(x); stack.append(x)
        return len(seen) == len(nodes)

# **Main SA loop (hot → anneal → cold)**
For every SA iteration:

**1- Proposal generation (propose_flip):**
Chooses a border edge, builds a two-district cluster, flips it.
This is the proposal step.

**2- Pre-checks:**

Non-empty district check: all(np.any(y_prop == d) for d in ...). If a district would become empty, reject immediately.

Contiguity check: check_contiguity(geo_edge, y_prop, num_districts). If non-contiguous, reject immediately. This prevents expensive simulation for obviously invalid proposals.

**3- Evaluation (simulation):**

Convert y_prop into an action matrix with labels_to_action.

Make tmp_env = deepcopy(env_work) and call tmp_env.step(action_prop). That runs step and reward logic to produce reward_prop and tmp_env.FrankenData (the hypothetical post-proposal state).

This tmp_env.step is where the proposal is actually evaluated (it runs opinion update, elects reps, computes augmented graph and reward).

**4- Acceptance (Metropolis-style):**

Compute diff = reward_prop - reward_curr.
Acceptance probability p_accept = min(1, exp(diff / T)), where T is the current temperature (annealed linearly from T_init → T_final).

With probability p_accept accept the proposal: commit by setting env_work = tmp_env and reward_curr = reward_prop.

If rejected, env_work remains unchanged.

In [ ]:
def simulated_annealing(
    env,
    hot_steps,
    anneal_steps,
    cold_steps,
    T_init,
    T_final,
    seed,
    eps_indiff , eps_assim ,eps_backfire , eps_irrel , eps_amb ,
    assim_shift , back_shift , indiff_shift , amb_shift , irr_shift
):

    rng = np.random.default_rng(seed)

    curr_labels = env.FrankenData.dist_label.clone().long().numpy().astype(int)
    print(type(curr_labels ))
    if np.any(curr_labels < 0):
        raise ValueError("Current FrankenData.dist_label contains -1 (unassigned). Assign before SA.")

    # Do a single step with the current labels to produce current opinion update & reward baseline
    action_curr = labels_to_action(curr_labels, env.num_districts)
    # print('curr_labels',curr_labels)
    _, reward_curr, _, _, _ = env.step(action_curr,eps_indiff , eps_assim ,eps_backfire , eps_irrel , eps_amb ,
                                                            assim_shift , back_shift , indiff_shift , amb_shift , irr_shift )   # env_work mutated here to the "post-step" state
    # print('step')
    ensemble_labels = [env.FrankenData.dist_label.clone().long().numpy().astype(int).copy()]
    ensemble_rewards = [float(reward_curr)]

    total_steps = hot_steps + anneal_steps + cold_steps
    step_idx = 0

    # print('Geo-edge')

    # Geo-edge in numpy form for contiguity checks
    geo_edge = env.FrankenData.geographical_edge.clone().numpy()

    # MAIN loop: hot -> anneal -> cold
    for phase, n_steps in (("hot", hot_steps), ("anneal", anneal_steps), ("cold", cold_steps)):
        for _ in range(n_steps):
            # print('loop')
            # T = temperature(step_idx)
            T = max(T_final, T_init + (T_final - T_init) * (step_idx / max(1, total_steps - 1)))
            step_idx += 1

            # propose
            curr_labels = env.FrankenData.dist_label.clone().long().numpy().astype(int)
            y_prop = propose_flip(curr_labels,geo_edge, rng)

            # quick pre-checks
            # - non-empty districts
            if not all(np.any(y_prop == d) for d in range(env.num_districts)):
                # reject immediately (keeps current plan)
                ensemble_labels.append(curr_labels.copy())
                ensemble_rewards.append(float(reward_curr))
                continue

            # - contiguity check: use  check_contiguity (fast pre-filter)
            # if not env.check_contiguity(geo_edge, y_prop, env.num_districts):
            #     # reject immediately
            #     # ensemble_labels.append(curr_labels.copy())
            #     # ensemble_rewards.append(float(reward_curr))
            #     env.reset()
            #     continue

            # Build action for proposal (one-hot rows)
            action_prop = labels_to_action(y_prop, env.num_districts)

            # Evaluate proposal by simulating one step
            _, reward_prop, _, _, _ = env.step(action_prop,eps_indiff = 0, eps_assim = 3,eps_backfire = 3, eps_irrel = 200, eps_amb = 0,
                                                            assim_shift = 1, back_shift = -1, indiff_shift =0, amb_shift = 0, irr_shift =0)   # uses  env.step / env.reward logic

            # Metropolis acceptance on rewards with temperature T:
            # p = min(1, exp((reward_prop - reward_curr)/T))
            diff = float(reward_prop) - float(reward_curr)
            # guard against overflow
            try:
                p_accept = math.exp(diff / max(1e-8, T))
                p_accept = min(1.0, p_accept)
            except OverflowError:
                p_accept = 1.0 if diff > 0 else 0.0

            if rng.random() < p_accept:
                # accept: commit tmp_env as the new working environment
                reward_curr = float(reward_prop)
                ensemble_labels.append(env.FrankenData.dist_label.clone().long().numpy().astype(int).copy())
                ensemble_rewards.append(reward_curr)
            else:
                # reject: keep current env
                env.reset()

    return env, ensemble_labels, ensemble_rewards
